In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Get our dataset

In [2]:

wine_data = load_wine()

In [3]:
dir(wine_data)

['DESCR', 'data', 'feature_names', 'frame', 'target', 'target_names']

# Look at its characteristics

In [4]:
print(wine_data.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

In [5]:
features = wine_data.data
target = wine_data.target
wine_df = pd.DataFrame(features,columns=wine_data.feature_names)



,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


# Train our model

In [7]:
X_train, X_test,y_train,y_test = train_test_split(wine_df,wine_data.target,test_size=0.3,random_state=42)

In [8]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import KFold, cross_val_score

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from xgboost.sklearn import XGBClassifier

In [9]:
# testing all the ensemble methods available to sklearn
models=[]
models.append(('AB', AdaBoostClassifier()))
models.append(('GBM', GradientBoostingClassifier()))
models.append(('ET', ExtraTreesClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('XGB',XGBClassifier()))

In [10]:
results = {}
accuracy = {}
for name, model in models:
    kfold = KFold(n_splits=10)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results[name] = (cv_results.mean(), cv_results.std())
    model.fit(X_train, y_train)
    _ = model.predict(X_test)
    accuracy[name] = accuracy_score(y_test, _)

In [11]:
results

{'AB': (0.887820512820513, 0.08883468309420073),
 'GBM': (0.9198717948717947, 0.05081922557815023),
 'ET': (0.9762820512820513, 0.049987671063799755),
 'RF': (0.9685897435897436, 0.051637390700375194),
 'XGB': (0.9525641025641025, 0.06222903127954666)}

In [12]:
accuracy

{'AB': 0.9259259259259259,
 'GBM': 0.9074074074074074,
 'ET': 1.0,
 'RF': 1.0,
 'XGB': 0.9444444444444444}

In [13]:
import matplotlib.pyplot as plt
best_model = ExtraTreesClassifier()
best_model.fit(X_train, y_train)
feature_importance = best_model.feature_importances_
 
# Normalizing the individual importances
feature_importance_normalized = np.std([tree.feature_importances_ for tree in
                                        best_model.estimators_],
                                        axis = 0)


alcohol 0.10004141812906875
malic_acid 0.05503066785868711
ash 0.02446827568899026
alcalinity_of_ash 0.05237086622935617
magnesium 0.04029384421338456
total_phenols 0.0802756666852924
flavanoids 0.12438061438946629
nonflavanoid_phenols 0.023044905791050067
proanthocyanins 0.04861831300381969
color_intensity 0.11633632327962189
hue 0.08122441236113437
od280/od315_of_diluted_wines 0.10256708362100582
proline 0.1097889025902571


# Persist Model

In [14]:
import joblib


joblib.dump(best_model, 'extratrees.joblib') 

['extratrees.joblib']